# Raw time of flight

In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pydoocs
import time
from scipy import signal
from IPython.display import HTML, display,clear_output
import ipywidgets as wg


In [ ]:
adc_doocs_address = 'FLASH.FEL/ADC.ADQ.BL1/EXP1.CH00/CH00.DAQ.TD'
time_axis = []
adc_trace_average = []
fitted_peaks = []
dropdown_values = [1,2]
train_ID = 0

In [ ]:
def create_dropdown_values():
    dropdown_values = np.around(time_axis[fitted_peaks[0]], decimals=2)
    ion_1_dropdown.options = dropdown_values
    ion_2_dropdown.options = dropdown_values

def plot_trace(axis, adc_trace_average, fitted_peaks, kind):
    if kind == 'time':
        x_label = 'time [u secs]'
        plot_num = 1
        if plt.fignum_exists(1):
            plt.close(1)
    elif kind == 'mass':
        x_label = 'm/z'
        plot_num = 2
        if plt.fignum_exists(2):
            plt.close(2)
    fig = plt.figure(figsize=(9, 8), num = plot_num)
    plt.plot(axis, adc_trace_average)
    plt.plot(axis[fitted_peaks[0]],fitted_peaks[1]['peak_heights'],'ro')
    plt.title("Average TOF")
    plt.ylabel('Intensity [a.u.]')
    plt.xlabel(x_label)
#     plt.show()
    
def print_peaks(axis, fitted_peaks, decimals = 3):
    data = [(axis[fitted_peaks[0]][index],fitted_peaks[1]['peak_heights'][index]) for index in range(len(axis[fitted_peaks[0]]))]
    data = np.around(data, decimals = decimals)
    display(HTML('<h3>Pos. Height</h3>'))
    display(HTML(
        '<table><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    ))
    
def plot_trace_and_print_peak_table(axis, adc_trace_average, kind = 'time'):
    global fitted_peaks
    fitted_peaks = signal.find_peaks(adc_trace_average, height=2)
    plot_trace(axis, adc_trace_average, fitted_peaks, kind)
    print_peaks(axis,fitted_peaks)
    create_dropdown_values()



In [ ]:
def create_fake_tof(length= 1000, peaks = 20):
    global time_axis, adc_trace_average
    time_axis = np.linspace(0,length/2, length)
    adc_trace_average = np.random.random(length)
    peak_pos = random.sample(range(5,length),peaks)
    peak_height = np.random.random(peaks)*10
    adc_trace_average[peak_pos] = peak_height
    return time_axis, adc_trace_average

def on_button_clicked_mock(b):
    with output_raw:
        clear_output(True)
        time_axis, adc_trace_average = create_fake_tof()
        plot_trace_and_print_peak_table(time_axis, adc_trace_average, kind = 'time')


In [ ]:
def wait_for_next_train():
    global train_ID
    while True:
        current_ID = pydoocs.read(adc_doocs_address)['macropulse']
        if train_ID != current_ID:
            train_ID = current_ID
            break
### if DOOCS server do not provide correct "macroppulse number"
#     seconds_between_trains = 0.1       # 10Hz FLASH repetion rate
#     time.sleep(seconds_between_trains)

def time_value_matrix_waiting(adc_doocs_address):
    global progress
    progress.value +=1 
    wait_for_next_train()
    adc_dictonary = pydoocs.read(adc_doocs_address)
    time_value_matrix = adc_dictonary['data']
    return time_value_matrix

def record_tof_average(adc_doocs_address):
    global time_axis, adc_trace_average, progress
    number_of_pulse_trains_to_average = number_of_pulses_to_record.value
    progress = wg.IntProgress(value = 0, min = 0, max = number_of_pulse_trains_to_average,  description='Recording:')
    display(progress)   
    time_value_matrices = [time_value_matrix_waiting(adc_doocs_address) for each_train in range(number_of_pulse_trains_to_average)]
    adc_traces = [each_matrix[:,1] for each_matrix in time_value_matrices]
    adc_trace_average = np.average(adc_traces, axis=0)
    time_axis = time_value_matrices[0][:, 0]
    return time_axis, adc_trace_average

def on_button_clicked_record(b):
    with output_raw:
        clear_output(True)
        time_axis, adc_trace_average = record_tof_average(adc_doocs_address)
        plot_trace_and_print_peak_table(time_axis, adc_trace_average, kind = 'time')


In [ ]:
def on_button_clicked_save(b):
    global time_axis, adc_trace_average
    np.savez('data/' + file_name.value + '.npz', time_axis = time_axis, adc_trace_average = adc_trace_average)
    
def on_button_clicked_load(b):
    global time_axis, adc_trace_average
    with output_raw:
        clear_output(True)
        npz_file = np.load('data/' + file_name.value + '.npz')
        time_axis, adc_trace_average = npz_file['time_axis'], npz_file['adc_trace_average'] 
        plot_trace_and_print_peak_table(time_axis, adc_trace_average, kind = 'time')


In [ ]:
number_of_pulses_to_record = wg.BoundedIntText(
    value=10,
    min=1,
    max=1000,
    step=1,
    description='',
    disabled=False
)

file_name = wg.Text(
    value='',
    placeholder='file name',
    description='',
    disabled=False
)

ion_1_dropdown = wg.Dropdown(
    options=dropdown_values,
    value=dropdown_values[0],
    description='TOF Ion 1:',
    disabled=False,
)

ion_2_dropdown = wg.Dropdown(
    options=dropdown_values,
    value=dropdown_values[1],
    description='TOF Ion 2:',
    disabled=False,
)

ion_1_m_over_z = wg.BoundedFloatText(
    value=5,
    min=1,
    max=500,
    step=0.01,
    description='m/z:',
    disabled=False
)

ion_2_m_over_z = wg.BoundedFloatText(
    value=10,
    min=1,
    max=500,
    step=0.01,
    description='m/z:',
    disabled=False
)

In [ ]:
button_record = wg.Button(description="Record")
button_load = wg.Button(description="Load")
button_save = wg.Button(description="Save")
button_mock = wg.Button(description="Mock")

button_record.on_click(on_button_clicked_record)
button_mock.on_click(on_button_clicked_mock)
button_save.on_click(on_button_clicked_save)
button_load.on_click(on_button_clicked_load)

output_raw = wg.Output()

upper_box = wg.HBox([button_record,number_of_pulses_to_record, button_mock])
lower_box = wg.HBox([button_load, button_save, file_name])

display( wg.VBox([upper_box, lower_box]), output_raw)


# Mass calibration 

<font size="4">
$
\frac{m}{z} = a \ \tau^2  + b \\
a = \frac{\frac{m}{z}_1 - \frac{m}{z}_2}{\tau_1^2 - \tau_2^2} \quad \quad 
b = \frac{m}{z}_1 - \frac{\frac{m}{z}_1 - \frac{m}{z}_2}{\tau_1^2 - \tau_2^2} \tau_1^2
$
</font>

In [ ]:
def mass_calibration(time_axis):
    m_over_z_ion_1 = ion_1_dropdown.value
    tof_ion_1 = ion_1_m_over_z.value
    m_over_z_ion_2 = ion_2_dropdown.value
    tof_ion_2 = ion_2_m_over_z.value
    a = (m_over_z_ion_1-m_over_z_ion_2)/(tof_ion_1**2-tof_ion_2**2)
    b = m_over_z_ion_1 - (m_over_z_ion_1-m_over_z_ion_2)/(tof_ion_1**2-tof_ion_2**2) * tof_ion_1**2
    return a * time_axis * time_axis + b

def on_button_clicked_mass_calibration(b):
    with output_mass_calibration:
        clear_output(True)
        plot_trace_and_print_peak_table(mass_calibration(time_axis), adc_trace_average, kind = 'mass')
            

In [ ]:
button_mass_calibration = wg.Button(description="calibrate")
output_mass_calibration = wg.Output()

left_box = wg.VBox([ion_1_dropdown,ion_2_dropdown])
right_box = wg.VBox([ion_1_m_over_z,ion_2_m_over_z])

display(wg.HBox([left_box, right_box, button_mass_calibration]), output_mass_calibration)

button_mass_calibration.on_click(on_button_clicked_mass_calibration)
